In [1]:
import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo
from ray.rllib.models import ModelCatalog
from environment_rllib import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions

import matplotlib.pyplot as plt

import tensorflow as tf

PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = False

def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

ModelCatalog.register_custom_model('my_model', DenseNetModelLarge)

# config = {"env": MyEnv,
#           "num_workers": NUM_WORKERS,
#           "num_gpus": NUM_GPUS,
#           "num_cpus_per_worker": NUM_CPUS_PER_WORKER,
#           "num_sgd_iter": NUM_SGD_ITER,
#           "lr": LEARNING_RATE,
#           "gamma": GAMMA,  # default=0.99
#           "model": {"custom_model": "my_model"}
#           # "framework": framework
#           }  # use tensorflow 2
config = {"env": MyEnv,"num_gpus": 0,"num_workers": 1, "num_cpus_per_worker": 0,"num_gpus": 0}
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    model_path = PROJECT + '/checkpoints/' + 'fwd_1/checkpoint_000351/checkpoint-351'
    trainer.restore(checkpoint_path=model_path)

models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
text_name = models_dir + TRIAL + '.txt'
with open(text_name, "w") as fp:
    trainer.get_policy().model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
png_name = models_dir + TRIAL + '.png'
plot_model(trainer.get_policy().model.base_model, to_file=png_name, show_shapes=True)

# Instanciate the evaluation env
eval_env = MyEnv({})

# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

2021-10-06 16:56:51,311	INFO services.py:1265 -- View the Ray dashboard at http://127.0.0.1:8265
2021-10-06 16:57:00,549	INFO trainer.py:714 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-10-06 16:57:00,549	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-10-06 16:57:00,550	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-10-06 16:57:06,773	WARNING util.py:55 -- Install gputil for GPU system monitoring.


('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


C:\Users\Takumi\anaconda3\envs\AI_Ray\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:

# Training & evaluation
record_mode = 1
results_dir = os.path.join('./' + PROJECT + '/results/')
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    results = trainer.train()

    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n----------------- Evaluation at steps:{steps} starting ! -----------------')
        print(pretty_print(results))
        check_point = trainer.save(checkpoint_dir=check_point_dir)
        win = 0
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            trajectory_length = 100
            env_blue_pos = [0]
            env_red_pos = [0]
            env_mrm_pos = [0]
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(eval_env.WINNDOW_SIZE_lon,eval_env.WINDOW_SIZE_lat))

            while not done:
                action_dict = {}
                for j in range(eval_env.blue_num):
                    if not eval_env.blue[j].hitpoint == 0:
                        action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])

                obs, rewards, dones, infos = eval_env.step(action_dict)
                done = dones["__all__"]
                #print(f'rewards:{rewards}')
                if record_mode == 0:
                    img = eval_env.render_movie(file_name,step_num)
                    video.write(img.astype('unit8'))
                elif record_mode == 1:
                    eval_env.render()
                elif record_mode == 2:
                    eval_env.render()
                    
                #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)
                
                #env_blue_pos.append(env_blue_pos_temp)
                #env_red_pos.append(env_red_pos_temp)
                #env_mrm_pos.append(env_mrm_pos_temp)
                #step_num = step_num + 1
                
                
            #del env_blue_pos[0]
            #del env_red_pos[0]
            #del env_mrm_pos[0]
            
            #hist_blue_pos = np.vstack(env_blue_pos)
            #hist_red_pos = np.vstack(env_red_pos)
            #hist_mrm_pos = np.vstack(env_mrm_pos)
            
            #f = open(results_file,'wb')
            #pickle.dump(emv_blue_pos,f)
            #pickle.dump(emv_red_pos,f)
            #pickle.dump(emv_mrm_pos,f)
            #f.close()
            
            if record_mode == 0:
                video.release()

ray.shutdown()

2021-10-06 16:57:10,405	WARNING deprecation.py:39 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!
2021-10-06 16:57:12,566	WARNING deprecation.py:39 -- DeprecationWarning: `compute_action` has been deprecated. Use `compute_single_action` instead. This will raise an error in the future!



----------------- Evaluation at steps:0 starting ! -----------------
agent_timesteps_total: 4000
custom_metrics: {}
date: 2021-10-06_16-57-12
done: false
episode_len_mean: 562.3333333333334
episode_media: {}
episode_reward_max: 3.8927000000000005
episode_reward_mean: 0.5288666666666701
episode_reward_min: -1.1783999999999923
episodes_this_iter: 3
episodes_total: 3
experiment_id: 0258895a28d64448a57f5261e765e07b
hostname: DESKTOP
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 2.0723531246185303
        entropy_coeff: 0.0
        kl: 0.007047607097774744
        model: {}
        policy_loss: -0.011758591048419476
        total_loss: 0.06090806424617767
        vf_explained_var: -0.31641122698783875
        vf_loss: 0.07125714421272278
  num_agent_steps_sampled: 4000
  num_agent_steps_trained: 4000
  num_steps_sampled: 4000
  num_steps_trained: 4000
iterat


----------------- Evaluation at steps:5 starting ! -----------------
agent_timesteps_total: 24000
custom_metrics: {}
date: 2021-10-06_16-58-50
done: false
episode_len_mean: 652.0588235294117
episode_media: {}
episode_reward_max: 3.8941500000000002
episode_reward_mean: -1.028549999999998
episode_reward_min: -2.1996000000000007
episodes_this_iter: 3
episodes_total: 17
experiment_id: 0258895a28d64448a57f5261e765e07b
hostname: DESKTOP
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 2.0125885009765625
        entropy_coeff: 0.0
        kl: 0.006026575341820717
        model: {}
        policy_loss: -0.011828646995127201
        total_loss: 0.04203527048230171
        vf_explained_var: -0.4901210069656372
        vf_loss: 0.052658602595329285
  num_agent_steps_sampled: 24000
  num_agent_steps_trained: 24000
  num_steps_sampled: 24000
  num_steps_trained: 24000



----------------- Evaluation at steps:10 starting ! -----------------
agent_timesteps_total: 44000
custom_metrics: {}
date: 2021-10-06_17-00-23
done: false
episode_len_mean: 696.8709677419355
episode_media: {}
episode_reward_max: 3.8941500000000002
episode_reward_mean: -1.1091354838709646
episode_reward_min: -2.1996000000000007
episodes_this_iter: 3
episodes_total: 31
experiment_id: 0258895a28d64448a57f5261e765e07b
hostname: DESKTOP
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 1.9267640113830566
        entropy_coeff: 0.0
        kl: 0.008496327325701714
        model: {}
        policy_loss: -0.01509306114166975
        total_loss: -0.002140073338523507
        vf_explained_var: 0.13809950649738312
        vf_loss: 0.011253723874688148
  num_agent_steps_sampled: 44000
  num_agent_steps_trained: 44000
  num_steps_sampled: 44000
  num_steps_trained: 440


----------------- Evaluation at steps:15 starting ! -----------------
agent_timesteps_total: 64000
custom_metrics: {}
date: 2021-10-06_17-01-54
done: false
episode_len_mean: 706.8222222222222
episode_media: {}
episode_reward_max: 3.906
episode_reward_mean: -0.8681599999999973
episode_reward_min: -2.1996000000000007
episodes_this_iter: 3
episodes_total: 45
experiment_id: 0258895a28d64448a57f5261e765e07b
hostname: DESKTOP
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 1.9124277830123901
        entropy_coeff: 0.0
        kl: 0.011132133193314075
        model: {}
        policy_loss: -0.023399489000439644
        total_loss: -0.01129877008497715
        vf_explained_var: -0.16688518226146698
        vf_loss: 0.00987429078668356
  num_agent_steps_sampled: 64000
  num_agent_steps_trained: 64000
  num_steps_sampled: 64000
  num_steps_trained: 64000
iterations

...


----------------- Evaluation at steps:450 starting ! -----------------
agent_timesteps_total: 1804000
custom_metrics: {}
date: 2021-10-06_19-12-10
done: false
episode_len_mean: 656.92
episode_media: {}
episode_reward_max: 3.9139500000000003
episode_reward_mean: 0.45860350000000233
episode_reward_min: -2.230000000000002
episodes_this_iter: 2
episodes_total: 1318
experiment_id: 0258895a28d64448a57f5261e765e07b
hostname: DESKTOP
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.30000001192092896
        cur_lr: 4.999999873689376e-05
        entropy: 1.0514553785324097
        entropy_coeff: 0.0
        kl: 0.00945884920656681
        model: {}
        policy_loss: -0.027965663000941277
        total_loss: -0.005753252189606428
        vf_explained_var: 0.017624225467443466
        vf_loss: 0.01937476173043251
  num_agent_steps_sampled: 1804000
  num_agent_steps_trained: 1804000
  num_steps_sampled: 1804000
  num_steps_trained: 1804


----------------- Evaluation at steps:455 starting ! -----------------
agent_timesteps_total: 1824000
custom_metrics: {}
date: 2021-10-06_19-13-32
done: false
episode_len_mean: 655.87
episode_media: {}
episode_reward_max: 3.9159000000000006
episode_reward_mean: 0.5319905000000024
episode_reward_min: -2.1819500000000005
episodes_this_iter: 3
episodes_total: 1333
experiment_id: 0258895a28d64448a57f5261e765e07b
hostname: DESKTOP
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.30000001192092896
        cur_lr: 4.999999873689376e-05
        entropy: 1.03117036819458
        entropy_coeff: 0.0
        kl: 0.010514600202441216
        model: {}
        policy_loss: -0.028470611199736595
        total_loss: -0.015947911888360977
        vf_explained_var: 0.3836665749549866
        vf_loss: 0.009368319064378738
  num_agent_steps_sampled: 1824000
  num_agent_steps_trained: 1824000
  num_steps_sampled: 1824000
  num_steps_trained: 182400


----------------- Evaluation at steps:460 starting ! -----------------
agent_timesteps_total: 1844000
custom_metrics: {}
date: 2021-10-06_19-14-57
done: false
episode_len_mean: 652.67
episode_media: {}
episode_reward_max: 3.9159000000000006
episode_reward_mean: 0.6632550000000024
episode_reward_min: -2.2007499999999993
episodes_this_iter: 3
episodes_total: 1349
experiment_id: 0258895a28d64448a57f5261e765e07b
hostname: DESKTOP
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.30000001192092896
        cur_lr: 4.999999873689376e-05
        entropy: 1.0122690200805664
        entropy_coeff: 0.0
        kl: 0.014070413075387478
        model: {}
        policy_loss: -0.026031630113720894
        total_loss: -0.010506493970751762
        vf_explained_var: 0.289519339799881
        vf_loss: 0.01130401249974966
  num_agent_steps_sampled: 1844000
  num_agent_steps_trained: 1844000
  num_steps_sampled: 1844000
  num_steps_trained: 184400


----------------- Evaluation at steps:465 starting ! -----------------
agent_timesteps_total: 1864000
custom_metrics: {}
date: 2021-10-06_19-16-43
done: false
episode_len_mean: 663.38
episode_media: {}
episode_reward_max: 3.9159000000000006
episode_reward_mean: 0.7144060000000023
episode_reward_min: -2.2007499999999993
episodes_this_iter: 3
episodes_total: 1363
experiment_id: 0258895a28d64448a57f5261e765e07b
hostname: DESKTOP
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.30000001192092896
        cur_lr: 4.999999873689376e-05
        entropy: 1.0405025482177734
        entropy_coeff: 0.0
        kl: 0.01550863403826952
        model: {}
        policy_loss: -0.03338352218270302
        total_loss: -0.010600099340081215
        vf_explained_var: 0.2509498596191406
        vf_loss: 0.0181308314204216
  num_agent_steps_sampled: 1864000
  num_agent_steps_trained: 1864000
  num_steps_sampled: 1864000
  num_steps_trained: 1864000



----------------- Evaluation at steps:470 starting ! -----------------
agent_timesteps_total: 1884000
custom_metrics: {}
date: 2021-10-06_19-18-16
done: false
episode_len_mean: 671.39
episode_media: {}
episode_reward_max: 3.9159000000000006
episode_reward_mean: 0.47430550000000227
episode_reward_min: -2.2007499999999993
episodes_this_iter: 2
episodes_total: 1377
experiment_id: 0258895a28d64448a57f5261e765e07b
hostname: DESKTOP
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.30000001192092896
        cur_lr: 4.999999873689376e-05
        entropy: 1.0871529579162598
        entropy_coeff: 0.0
        kl: 0.01084133516997099
        model: {}
        policy_loss: -0.03509129211306572
        total_loss: -0.015090832486748695
        vf_explained_var: 0.06174226105213165
        vf_loss: 0.016748061403632164
  num_agent_steps_sampled: 1884000
  num_agent_steps_trained: 1884000
  num_steps_sampled: 1884000
  num_steps_trained: 1884